#  create NOAA DSDP taxa list for import
## 1-96 taxa

create taxa list to import in database

In [3]:
import sys
import csv
import glob
import os
import requests
import re
import time 

sys.path.append('../../')
import pandas as pd
import numpy as np

# import db 
import scripts.normalize_taxa as nt
from config import OUTPUT_DIR, CLEAN_DATA_DIR, RAW_DATA_DIR
from scripts.pbdb import get_parent_taxa, PBDB_TAXA_ID, PBDB_TAXA_NAME
from scripts.shared_utils import (
    log_df
)
import scripts.pbdb as pbdb

In [4]:
date='2024-04-03'
input_file_species_temp = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_species_temp_{date}.csv"


date='2024-06-06'
input_file_species = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_species_{date}.csv"
crosswalk_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_crosswalk_{date}.csv"
taxa_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_list_{date}.csv"


## clean up columns in input species file

In [5]:
PI_df = pd.read_csv(input_file_species_temp, dtype=str)
log_df(PI_df)
# (7763, 54)

(7763, 54)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,pbdb_taxon_rank_y,kingdom_taxon_id_y,kingdom_taxon_name_y,phylum_taxon_id_y,phylum_taxon_name_y,genus_taxon_id_y,genus_taxon_name_y,species_taxon_id,species_taxon_name,phylum match
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cols = [col for col in PI_df.columns if not col.endswith('_y')]
cols = [col for col in cols if col not in ['genus species', 'phylum match', 'pbdb_taxon_id_old','pbdb_taxon_name_old', 'pbdb_taxon_rank_old' ]]
cols

['taxon_group',
 'verbatim_name',
 'name',
 'Comment',
 'Notes (change to Internal only notes?)',
 'Any taxon above genus modifier',
 'Any taxon above genus',
 'genus modifier',
 'genus name',
 'subgenera modifier',
 'subgenera name',
 'species modifier',
 'species name',
 'subspecies modifier',
 'subspecies name',
 'non-taxa descriptor',
 'comments',
 'Corrections to pbdb_taxon_rank',
 'pbdb_taxon_id',
 'pbdb_taxon_name',
 'pbdb_taxon_rank',
 'Corrections to pbdb_taxon',
 'family_taxon_id',
 'family_taxon_name',
 'superfamily_taxon_id',
 'superfamily_taxon_name',
 'order_taxon_id',
 'order_taxon_name',
 'class_taxon_id',
 'class_taxon_name',
 'phylum_taxon_id',
 'phylum_taxon_name',
 'kingdom_taxon_id',
 'kingdom_taxon_name',
 'unranked clade_taxon_id',
 'unranked clade_taxon_name',
 'genus_taxon_id',
 'genus_taxon_name',
 'species_taxon_id',
 'species_taxon_name']

In [7]:
updated_df = PI_df[cols].copy()
updated_df.shape
# (7763, 40)

(7763, 40)

In [8]:
updated_df[updated_df['pbdb_taxon_id'] == updated_df['species_taxon_id']].shape

(0, 40)

In [10]:
updated_df.loc[updated_df['species_taxon_id'].notna(), 'pbdb_taxon_id'] = updated_df['species_taxon_id']
updated_df.loc[updated_df['species_taxon_id'].notna(), 'pbdb_taxon_name'] = updated_df['species_taxon_name']
updated_df.loc[updated_df['species_taxon_id'].notna(), 'pbdb_taxon_rank'] = 'species'

In [11]:
updated_df[updated_df['pbdb_taxon_id'] == updated_df['species_taxon_id']].shape

(670, 40)

In [ ]:
updated_df.to_csv(input_file_species, index=False)

## create crosswalk file

In [12]:
PI_df = pd.read_csv(input_file_species, dtype=str)
log_df(PI_df)
# (7763, 40)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN


In [13]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 41)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN,Alabamina haitiensis


In [14]:
crosswalk_df = nt.create_taxa_crosswalk_df(df)
crosswalk_df['eodp_id'] = crosswalk_df.index
crosswalk_df['eodp_id'] = 'NOAA-1-96:' + crosswalk_df['eodp_id'].astype(str)

log_df(crosswalk_df)
# 7763
# 7763
# 7762

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'verbatim_name', 'name comment field', 'Comment', 'Notes (change to Internal only notes?)', 'comments']
initial df:  (7763, 17)
remove nontaxa df:  (7763, 17)
drop duplicates df:  (7762, 17)
(7762, 18)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments,eodp_id
0,,,Abyssamina,,,,incisa,,,,Abyssamina incisa,benthic_forams,Abyssamina incisa,NaN,,,,NOAA-1-96:0
1,,,Adercotryma,,,,glomeratum,,,,Adercotryma glomeratum,benthic_forams,Adercotryma glomeratum,NaN,,,,NOAA-1-96:1
2,,,Adercotryma,,,,sp.,,,,Adercotryma sp.,benthic_forams,Adercotryma sp.,NaN,,,,NOAA-1-96:2
3,,,Alabamina,,,,decorata,,,,Alabamina decorata,benthic_forams,Alabamina decorata,NaN,,,,NOAA-1-96:3
4,,,Alabamina,,,,haitiensis,,,,Alabamina haitiensis,benthic_forams,Alabamina haitiensis,NaN,,,,NOAA-1-96:4


In [ ]:
crosswalk_df.to_csv(crosswalk_file, index=False)

## create taxa file

In [15]:
PI_df = pd.read_csv(input_file_species, dtype=str)
log_df(PI_df)
# (7763, 40)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,288974,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN


In [16]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 41)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,species_taxon_id,species_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,762,Abyssamina,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,774,Adercotryma,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,Foraminifera,212476,Rhizaria,NaN,NaN,788,Alabamina,NaN,NaN,Alabamina haitiensis


In [17]:
taxa_df = nt.create_taxa_list_df(df)
log_df(taxa_df)
# 7763
# 7763
# 7656

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id', 'family_taxon_name', 'order_taxon_id', 'order_taxon_name', 'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name']
initial df:  (7763, 25)
remove nontaxa df:  (7763, 25)
drop duplicates df:  (7656, 25)
(7656, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Abyssamina,,,,incisa,,,,...,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria
1,,,Adercotryma,,,,glomeratum,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,,,Adercotryma,,,,sp.,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
3,,,Alabamina,,,,decorata,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria
4,,,Alabamina,,,,haitiensis,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria


In [ ]:
taxa_df.to_csv(taxa_file, index=False)

## QA taxon groups

In [26]:
taxa_df = pd.read_csv(taxa_file)
log_df(taxa_df)

(7656, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,NaN,NaN,Abyssamina,NaN,NaN,NaN,incisa,NaN,NaN,NaN,...,103796.0,Chilostomellidae,279579.0,Rotaliida,428504.0,Globothalamea,288974.0,Foraminifera,212476.0,Rhizaria
1,NaN,NaN,Adercotryma,NaN,NaN,NaN,glomeratum,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria
2,NaN,NaN,Adercotryma,NaN,NaN,NaN,sp.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria
3,NaN,NaN,Alabamina,NaN,NaN,NaN,decorata,NaN,NaN,NaN,...,82213.0,Alabaminidae,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria
4,NaN,NaN,Alabamina,NaN,NaN,NaN,haitiensis,NaN,NaN,NaN,...,82213.0,Alabaminidae,NaN,NaN,NaN,NaN,288974.0,Foraminifera,212476.0,Rhizaria


In [34]:
taxa_df['taxon_group'].unique()

array(['benthic_forams', 'diatoms', 'dinoflagellates', 'radiolarians',
       'ebridians', 'nannofossils', 'ostracods', 'phytoliths',
       'planktic_forams', 'pollen', 'silicoflagellates'], dtype=object)

In [33]:
taxa_group_df = taxa_df[['normalized_name', 'taxon_group']]
log_df(taxa_group_df)

(7656, 2)


,normalized_name,taxon_group
0,Abyssamina incisa,benthic_forams
1,Adercotryma glomeratum,benthic_forams
2,Adercotryma sp.,benthic_forams
3,Alabamina decorata,benthic_forams
4,Alabamina haitiensis,benthic_forams
